In [1]:
from astroquery.utils.tap.core import Tap
import pyvo

In [2]:
service = Tap(url="https://tapvizier.cds.unistra.fr/TAPVizieR/tap")
query = """
SELECT * 
FROM "V/165".igapsdr1
WHERE emitter >= 1
AND GLON BETWEEN 30 AND 215
AND GLAT BETWEEN -5 AND 5
"""

In [3]:
# Enviar consulta asíncrona (sin timeout)
job = service.submit_job(query)
job.run()  # El servidor procesa en segundo plano


AttributeError: 'Tap' object has no attribute 'submit_job'

In [ ]:
lon_min = 30
lon_max = 215
step = 5  # Segmentos de 5° con solapamiento
glon_ranges = np.arange(lon_min, lon_max, step)  # No superar 215°

In [ ]:
tables = []
for i, glon_start in enumerate(glon_ranges):
    glon_end = min(glon_start + step + 0.1, lon_max)  # Solapamiento de 0.1°
    print(f"Segmento {i+1}/{len(glon_ranges)}: l = {glon_start} a {glon_end}")
    
    max_retries = 3
    for attempt in range(max_retries):
        try:
            catalog = v.query_constraints(
                catalog="V/165",
                GLON=f"{glon_start} .. {glon_end}",
                GLAT="-5 .. 5"
            )
            if catalog:
                tables.append(catalog[0])
            time.sleep(2)
            break  # Éxito, salir del bucle de reintentos
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Error en segmento {glon_start}-{glon_end}: {str(e)}")
            time.sleep(10)